## Imports

In [1]:
import scipy.io
import numpy as np
from scipy.stats import linregress
from statsmodels.sandbox.stats.multicomp import multipletests 

# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

## Load dataset

In [2]:
mat = scipy.io.loadmat('sand.mat')
X = mat['X']
y = mat['Y'].ravel()

[n, p] = X.shape

### 3 Perform univariate feature selection for the sand data using:

> (a) Bonferroni correction to control the family-wise error rate(FWER). Use FWER = 0.05.

In [3]:
PValues = np.zeros(p)
Xsub = np.zeros(p)

for j in range(p):
    Xsub = X[:,j]
    # Use the stats models linear regression, since p value already is included
    # Otherwise check https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
    # Which explains how to expand the class in sklearn to calculate it
    slope, intercept, r_value, PValues[j], std_err = linregress(Xsub, y)

# Sort p-values in acending order
idx1 = np.argsort(PValues)
p = PValues[idx1]

remaining_features_bonf = len(np.where(p < (0.05 / 2016))[0]) # Amount af features included
print(f'Remaining features after correcting with Bonferroni correction: {remaining_features_bonf}.')

Remaining features after correcting with Bonferroni correction: 72.


> (b) Benjamini-Hochberg’s algorithm for FDR. Use an acceptable fraction of mistakes,
q = 0.15.

In [4]:
FDR = multipletests(PValues, alpha = 0.05, method = "fdr_bh")[1] # Computing Benjamini Hochberg's FDR

idx2 = np.argsort(FDR)
fdr = FDR[idx2]

remaining_features_fdr = len(np.where(fdr < 0.15)[0]) # How many values are below 0.15?
print(f'Remaining features after applying DFR: {remaining_features_fdr}.')

Remaining features after applying DFR: 721.


Compare the solutions in terms of number of selected features and selected features.

*It is clear that FDR "allows" for more features to be kept in the model, and through this the chance of having false discoveries are higher, this is done to make sure that all significant features are kept in the model, whereas bonferroni might remove some significant features because of the more stringent cutoff.*